In [1]:
import sys
sys.path.append('/home/rustem/projs/strategies')

import pandas as pd
import matplotlib.pyplot as plt
from src.utils import TradingSystem
from tqdm import tqdm

In [2]:
params = {
    'ma_window_width': 5,
    'q_std_rolling_width': 20,
    'sigma_coeff_for_SL': 0.1,
    'q': .8,
    'q_window_width': 20
}
ts = TradingSystem(**params)

df = pd.read_csv(
    '/home/rustem/projs/strategies/data/raw/BTC-USDT.csv',
    index_col=0)[['ts', 'o', 'h', 'l', 'c', 'vol']].sort_values(by='ts')

rows_list = []
history = 43
indices = range(0, df.shape[0] - history)[:500]
for i in tqdm(indices, total=len(indices)):
    data = df.iloc[i:i+history]

    signal = ts.get_signal(data)
    sl = ts.get_stoploss()
    is_reached_sl = ts.reached_sl()
    profit = ts.profit()

    last = data.iloc[-1].copy()
    last['signal'] = signal
    last['sl'] = sl
    last['is_reached_sl'] = is_reached_sl
    last['profit'] = profit
    rows_list.append(last.to_dict())

100%|██████████| 500/500 [00:02<00:00, 191.74it/s]


In [3]:
df = pd.DataFrame(rows_list)